In [12]:
import pandas as pd
import numpy as np

In [13]:
chitchat_df = pd.read_pickle("./chitchat.pkl")

In [20]:
chitchat_df.head()

,query,response,prompt
0,Hello,whats up MD,What's one thing your best friend doesn't know...
1,How are you doing today?,im doing good,What's one thing your best friend doesn't know...
2,whats up MD,"Im alright, I just took a nap. But it was one ...",What's one thing your best friend doesn't know...
3,"Im alright, I just took a nap. But it was one ...",oh wow haha,What's one thing your best friend doesn't know...
4,oh wow haha,Yeah,What's one thing your best friend doesn't know...


In [15]:
chitchat_df.shape

(166168, 3)

In [16]:
cc_new = pd.DataFrame().assign(X=chitchat_df['query'])

In [18]:
cc_new["Y"] = 0

In [19]:
cc_new.head()

,X,Y
0,Hello,0
1,How are you doing today?,0
2,whats up MD,0
3,"Im alright, I just took a nap. But it was one ...",0
4,oh wow haha,0


In [21]:
reddit_df = pd.read_pickle("./reddit.pkl")

In [37]:
reddit_df.head()

,author,created_at,full_link,id,selftext,subreddit,title,topic,is_submission,body,parent_id,permalink,parent_body
0,nightnightgame,2022-03-02T07:32:08Z,https://www.reddit.com/r/explainlikeimfive/com...,t4z4m7,When it has nothing to do with academics? It s...,explainlikeimfive,ELI5 why sports in American educational instit...,Politics,True,NaN,NaN,NaN,NaN
1,vcic502,2022-03-01T15:52:59Z,https://www.reddit.com/r/explainlikeimfive/com...,t4i1iw,I don't believe in any religion although I don...,explainlikeimfive,ELI5 What is Agnostic Atheism?,Politics,True,NaN,NaN,NaN,NaN
2,zedd1704,2022-02-26T05:14:38Z,https://www.reddit.com/r/explainlikeimfive/com...,t1tvdg,,explainlikeimfive,"Eli5: what is left, central and right in polit...",Politics,True,NaN,NaN,NaN,NaN
3,Vicarious_Paradox,2022-02-20T23:46:51Z,https://www.reddit.com/r/explainlikeimfive/com...,sxkv61,I thought the IOC was supposed to be neutral a...,explainlikeimfive,ELI5: Why is Taiwan called Chinese Taipei at t...,Politics,True,NaN,NaN,NaN,NaN
4,PerseusUzumacki,2022-02-12T00:01:50Z,https://www.reddit.com/r/explainlikeimfive/com...,sqjgol,,explainlikeimfive,ELI5 What does right and left mean in politics?,Politics,True,NaN,NaN,NaN,NaN


In [56]:
reddit_df.loc[reddit_df['is_submission'] == True]["selftext"]

0      When it has nothing to do with academics? It s...
1      I don't believe in any religion although I don...
2                                                       
3      I thought the IOC was supposed to be neutral a...
4                                                       
                             ...                        
294                                                     
295                                                     
296                                                     
297                                                     
298                                                     
Name: selftext, Length: 4276, dtype: object

In [71]:
reddit_df.loc[reddit_df['is_submission'] == False]["body"]

0                                                    ??
1     That’s what I thought. I’m sorry things haven’...
2            Nah i really don't care enough about this.
3                Let’s hear your counter-argument then.
4     Yeah i don't support the premise anyways: i do...
                            ...                        
33                         Without access to water tho?
34    People have been on this earth for thousands o...
35    Dubai is an exercise in excess and hubris. Wit...
36    To add on to your answer: For the places that ...
37    Not really. Populations in those areas were mu...
Name: body, Length: 40022, dtype: object

In [57]:
# reddit_df['selftext'].isna().sum()
val = reddit_df['body'].isna().sum()
val

4276

In [59]:
reddit_new = pd.DataFrame().assign(X=reddit_df['selftext'])


In [68]:
reddit_new.drop(['X'], axis = 1)

""
0
1
2
3
4
...
33
34
35
36


In [73]:
body = []
selftext = []
for index, row in reddit_df.iterrows():
    if row['is_submission'] == True:
        selftext.append(row["selftext"])
    else:
        body.append(row["body"]) 

In [82]:
l = selftext + body

In [84]:
reddit_new = pd.DataFrame({"X": l})

In [90]:
reddit_new["Y"] = 1

In [93]:
res_df = pd.concat([cc_new, reddit_new])

In [104]:
res_df

,X,Y
0,Hello,0
1,How are you doing today?,0
2,whats up MD,0
3,"Im alright, I just took a nap. But it was one ...",0
4,oh wow haha,0
...,...,...
44293,Without access to water tho?,1
44294,People have been on this earth for thousands o...,1
44295,Dubai is an exercise in excess and hubris. Wit...,1
44296,To add on to your answer: For the places that ...,1


In [98]:
res_df.to_csv(r'A:\MS2021\SUNY BUFFALO\Third sem\Info Retrieval\Project 4\ir_project_4\backend\classifier\data.csv', header=None, index=None, sep=' ', mode='a')